# Lesson 1: Simple ReAct Agent from Scratch

In [1]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [2]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [5]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [6]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [7]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_gpa:
e.g. calculate_gpa: A, B+, C
Calculates the GPA based on a comma-separated list of grades.

get_due_date:
e.g. get_due_date: "Research Paper"
Looks up the due date for a specific assignment.

define_term:
e.g. define_term: "Quantum Entanglement"
Provides a simple definition for a given term.

Example session:

Question: What is the due date for the Final Presentation?
Thought: I need to use get_due_date to find the due date for "Final Presentation".
Action: get_due_date: "Final Presentation"
PAUSE

You will be called again with this:

Observation: The Final Presentation is due on 2025-12-15

You then output:

Answer: The Final Presentation is due on 2025-12-15.
""".strip()


In [14]:
def calculate_gpa(grades_string):
    try:
        grade_map = {
            "A+": 9.0, "A": 8.0, "O": 10.0,
            "B+": 7.0, "B": 6.0,
            "C+": 5.0, "C": 2.0,
            "AB": 0.0, "D": 1.0,
            "F": 0.0
        }
        
        grades_list = [g.strip().upper() for g in grades_string.split(',')]
        points_total = sum(grade_map.get(grade, 0) for grade in grades_list)
        if len(grades_list) == 0:
            return "No grades provided, cannot calculate GPA."
        return f"Calculated GPA is {points_total / len(grades_list):.2f}"
    except Exception as e:
        return f"Error calculating GPA: {e}"

def get_due_date(assignment_name):
    due_dates = {
        "final paper": "2025-12-10",
        "midterm exam": "2025-10-25",
        "group project": "2025-11-20",
        "reading quiz": "2025-09-05"
    }
    assignment_name_lower = assignment_name.lower().strip()
    return f"The due date for '{assignment_name}' is {due_dates.get(assignment_name_lower, 'not found')}"

def define_term(term):
    definitions = {
        "procrastination": "The action of delaying or postponing something.",
        "synergy": "The interaction or cooperation of two or more agents to produce a combined effect greater than the sum of their separate effects.",
        "plagiarism": "The practice of taking someone else's work or ideas and passing them off as one's own."
    }
    term_lower = term.lower().strip()
    return f"Definition for '{term}': {definitions.get(term_lower, 'term not found')}"

known_actions = {
    "calculate_gpa": calculate_gpa,
    "get_due_date": get_due_date,
    "define_term": define_term
}

In [15]:
student_agent = Agent(prompt)

In [16]:
question = "What is my GPA if I got an A, a B+, and a C?"
print(f"\nUser's question: {question}")


User's question: What is my GPA if I got an A, a B+, and a C?


In [17]:
thought_action_response = student_agent(question)

In [18]:
grades = "A, B+, C"
observation = calculate_gpa(grades)
next_prompt = f"Observation: {observation}"
print(f"\nSimulating tool use: {next_prompt}")


Simulating tool use: Observation: Calculated GPA is 5.67


In [19]:
# Step 3: We feed the observation back to the agent
final_answer = student_agent(next_prompt)

# Print the final result
print(f"\nFinal Answer: {final_answer}")


Final Answer: Answer: The calculated GPA for the grades A, B+, and C is 5.67.


### Add loop 

In [20]:
question = "When is the midterm exam due?"
print(f"User's question: {question}")

User's question: When is the midterm exam due?


In [22]:
thought_action_response = student_agent(question)
assignment_name = "midterm exam"
observation = get_due_date(assignment_name)
next_prompt = f"Observation: {observation}"
print(f"\nSimulating tool use: {next_prompt}")
final_answer = student_agent(next_prompt)
print(f"\nFinal Answer: {final_answer}")


Simulating tool use: Observation: The due date for 'midterm exam' is 2025-10-25

Final Answer: Answer: The midterm exam is due on 2025-10-25.
